# Prompt Engineering with OpenAI and LangChain

## What is prompt engineering

Prompt engineering refers to the process of designing and refining prompts to achieve better results when using language models like GPT-3.5. It involves carefully crafting the input instructions or questions provided to the model to elicit the desired output. By understanding how to structure and frame prompts effectively, users can optimize the model's performance and obtain more accurate and useful responses.

Prompt engineering techniques typically involve several strategies:

- **Clear instructions**: Providing explicit and specific instructions in the prompt helps guide the model towards the desired output. For example, specifying the format of the answer or asking the model to think step by step before providing a final response.

- **Context framing**: Setting the context for the prompt can help the model understand the desired domain or topic. Including relevant information or mentioning a specific scenario can improve the accuracy of the generated response.

- **System messages**: Incorporating system-level instructions within the prompt can guide the model's behavior. For instance, instructing the model to consider the pros and cons before providing an opinion or asking it to generate a longer and more detailed response.

- **Iterative refinement**: Adjusting and fine-tuning the prompts based on the model's initial output is an iterative process. By analyzing the generated responses, users can identify shortcomings or biases and refine the prompts to address those issues.

Prompt engineering is an important aspect of using language models effectively. Through thoughtful design and refinement of prompts, users can enhance the quality, relevance, and usefulness of the generated responses.

### Importing OpenAI and reading the OpenAI API key from the envirnoment variables


In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

We create a helper function `get_completion()` which uses ChatGPT (gpt-3.5-turbo) model as default. The function takes user prompt as input and returns the response from the model

In [2]:
MODEL = "gpt-3.5-turbo"

In [3]:
def get_completion(prompt, model=MODEL):
    """Function to call the OpenAI API to get a completion of the prompt using ChatGPT model

    Args:
        prompt (str): user prompt to be completed
        model (str, optional): _description_. Defaults to "gpt-3.5-turbo".

    Returns:
        _type_: response from the ChatGPT model
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

We also create another helper function `create_notebook()`, which will be used later to create a new notebook with the response from the Model

In [4]:
def create_notebook(result, filename):
    
    allowed_chars = [' ', '{', '\n']
    
    directory = "generated_notebooks"
    if not filename.endswith('.ipynb'):
        filename = filename + '.ipynb'
    filepath = os.path.join(os.getcwd(), directory, filename)
    
    if len(result.split('```')) > 1:
        data = result.split('```')[1]
        if data[0] not in allowed_chars:
            data = data[4:]
    elif len(result.split('```')) == 1:
        data = result.split('```')[0]
        
    print('Creating notebook: '+ os.path.join(directory, filename))
    # data = result.split('```')[1]
    with open(filepath, 'w') as f:
         f.write(data)

We will now read a jupyter notebook as a json. The Notebook lists some tasks to be completed.
We will ask the ChatGPT to read the jupyter notebook and complete the tasks

#### The Input Notebook

![](images/task_nb.png)

In [5]:
from pprint import pprint
import json

with open("test_file.ipynb", "r") as f:
    data = json.load(f)

pprint(data)

{'cells': [{'cell_type': 'markdown',
            'metadata': {},
            'source': ['## Introduction']},
           {'cell_type': 'markdown',
            'metadata': {},
            'source': ['### Your task is to\n',
                       ' - Read the csv file into a pandas dataframe.\n',
                       ' - Find the list of unique cities where matches were '
                       'played\n',
                       ' - Find the columns which contains null values if any '
                       '?\n',
                       ' - List down top 5 most played venues\n',
                       ' - Get top 5 goal scorers of the tournament\n']}],
 'metadata': {'kernelspec': {'display_name': 'Python 3 (ipykernel)',
                             'language': 'python',
                             'name': 'python3'},
              'language_info': {'codemirror_mode': {'name': 'ipython',
                                                    'version': 3},
                                

### Prompting with OpenAI

Now we create a prompt which has a task to be followed. It also tells the model that what the input questions it has to work on

#### Prompt #1

In [6]:
prompt = f"""
You are a Data Scientist. Your task is listed in the
json input which will be followed by ```. The questions 
which are to be answered are under "cell_type":"markdown". 
Your output should be a step by step jupyter notebook with 
comments for the code in python.

The input: ```{data}```
"""

response = get_completion(prompt)
print(response)


## Solution

First, we need to import the necessary libraries.

```python
import pandas as pd
```

### Reading the CSV file into a pandas dataframe

We can use the `read_csv()` function from pandas to read the CSV file into a dataframe.

```python
df = pd.read_csv('matches.csv')
```

### Finding the list of unique cities where matches were played

We can use the `unique()` function to get the unique values in a column. In this case, we want to get the unique cities where matches were played. The city column in the dataframe is 'city'.

```python
unique_cities = df['city'].unique()
print(unique_cities)
```

### Finding the columns which contains null values if any

We can use the `isnull()` function to check for null values in the dataframe. We can then use the `any()` function to check if any of the columns contain null values.

```python
null_columns = df.columns[df.isnull().any()]
print(null_columns)
```

### Listing down top 5 most played venues

We can use the `value_counts()` func

We see that the output is just a python code which is not in the correct format in which we asked for, i.e. the jupyter notebook format

#### Prompt #2

In [7]:
prompt = f"""
You are a Data Scientist. Your task is listed in the
json input which will be followed by ```. The questions 
which are to be answered are under "cell_type":"markdown". 

Your output should be a step by step jupyter notebook in "json" format.
Add comments for the python output in "cell_type":"markdown" of the jupyter notebook.
    
Only and only output python code in json format of jupyter notebook.

The input: ```{data}```
"""
response = get_completion(prompt)
print(response)

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Introduction"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Your task is to\n",
    " - Read the csv file into a pandas dataframe.\n",
    " - Find the list of unique cities where matches were played\n",
    " - Find the columns which contains null values if any ?\n",
    " - List down top 5 most played venues\n",
    " - Get top 5 goal scorers of the tournament\n"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "#importing necessary libraries\n",
    "import pandas as pd\n",
    "\n",
    "#reading the csv file into a pandas dataframe\n",
    "df = pd.read_csv('matches.csv')\n",
    "\n",
    "#finding the list of unique cities where matches were played\n",
    "unique_cities = df['city'].unique()\n",
    "print('List of unique cities where matches were played:')\n",
    "print(unique_cities)\n",
    "\n",
    "#finding the columns 

 Prompt #1         |  Prompt #2
:-------------------------:|:-------------------------:
![](images/prompt1.png)    |  ![](images/prompt2.png)

Nicely done this time. We get the output in the way we wanted. 
Now we can save this response using our helper function `create_notebook()`

In [8]:
create_notebook(result=response, filename="prompt_with_openai")

Creating notebook: generated_notebooks/prompt_with_openai.ipynb


## Implementing with LangChain

In [9]:
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [10]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    HumanMessage,
    AIMessage,   ## not used in this notebook
    SystemMessage ## not used in this notebook
)

LangChain gives us flexibility to use different prompt templates. 
These templates can be used for Few-Shot learning as well. That means we can provide some examples to the model before it actually answers user questions. In this notebook we are going to use One-Shot Learning.

In [11]:
## This is a json format of a jupyter notebook
## It is used as an example for One-Shot learning to the Model
## This helps us to guide the model to the path we actually want it to follow

response_example = """{
 "cells": [
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Reading csv with Pandas"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "\n",
    "# Reading the csv file into a pandas dataframe\n",
    "matches_df = pd.read_csv('matches.csv')"
   ]
  }
 ]
}
"""

- **SystemMessagePromptTemplate** - It provides a default message to the model, that what it's actual task is
- **HumanMessagePromptTemplate** - It gives the model a prompt template from the users it can expect.
- **AIMessagePromptTemplate** - It tells the model what kind of response it should give to the user's question.
- **ChatPromptTemplate** - It combines all the Prompt Templates together. This returns a PromptValue, which we can convert to a string or Message object, depending on whether we want to use the formatted value as input to an llm or chat model.

In [12]:
## Defining a system message for the model. 
## The model should be able to understand that what it has to do is to solve a problem
## and provide a solution.
system_template = SystemMessagePromptTemplate.from_template(
    'You are a helpful assistant. You solve python problems and provide solutions '
    'in a jupyter notebook ipynb format when asked for.'
)

## Defining a human message and AI message for the model. 
## HumanMessage is the example user message for one-shot learning
## AIMessage is the example AI's response for one-shot learning
human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response}")

# create the list of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])

# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    input="Hi AI, How to read a csv file with pandas using jupyter notebook json format with comments",
    response=response_example
)

## we can use the promptValue object and use it as a string or a message, depending on the use case.
chat_prompt_value.to_messages()

[SystemMessage(lc_kwargs={'content': 'You are a helpful assistant. You solve python problems and provide solutions in a jupyter notebook ipynb format when asked for.', 'additional_kwargs': {}}, content='You are a helpful assistant. You solve python problems and provide solutions in a jupyter notebook ipynb format when asked for.', additional_kwargs={}),
 HumanMessage(lc_kwargs={'content': 'Hi AI, How to read a csv file with pandas using jupyter notebook json format with comments', 'additional_kwargs': {}}, content='Hi AI, How to read a csv file with pandas using jupyter notebook json format with comments', additional_kwargs={}, example=False),
 AIMessage(lc_kwargs={'content': '{\n "cells": [\n  {\n   "attachments": {},\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n    "## Reading csv with Pandas"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": null,\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "import pandas as pd\n",\n    "\n",\n    

We create a list of messages to keep track of the conversation between the user and the model

In [13]:
messages = chat_prompt_value.to_messages()

We will again use the same example of the Jupyter Notebook (`test_file.ipynb`) with the listed task

#### Prompt #1

In [14]:
## We create an input user prompt for the model.

input_prompt = f"""
You are a Data Scientist. Your tasks are listed in the
json input which will be followed by ```. The questions 
which are to be answered are under "cell_type":"markdown". 

The output should be under "cell_type":"code" and the comments 
of the code should be under "cell_type":"markdown" of the jupyter notebook.

```{data}```
"""

We first need to append the input to the list of messages for the Model. We use `HumanMessage` because it is the input prompt given by the user

In [15]:
messages.append(
    HumanMessage(content=input_prompt)
)

In [16]:
res = chat(messages)

print(res.content)

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Reading csv with Pandas"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importing necessary libraries\n",
    "import pandas as pd\n",
    "\n",
    "# Reading the csv file into a pandas dataframe\n",
    "matches_df = pd.read_csv('matches.csv')\n",
    "\n",
    "# Finding the list of unique cities where matches were played\n",
    "unique_cities = matches_df['city'].unique()\n",
    "print('List of unique cities where matches were played:')\n",
    "print(unique_cities)\n",
    "\n",
    "# Finding the columns which contains null values if any\n",
    "null_columns = matches_df.columns[matches_df.isnull().any()]\n",
    "print('Columns containing null values:')\n",
    "print(null_columns)\n",
    "\n",
    "# Listing down top 5 most played venues\n",
    "top_venues = matches_df['venue'].value_counts().head()\n",
    "p

We do not get the desired output format from the model, however the models response in solving the tasks are still correct.

Now we will continue to do the prompts iteratively, but this time in a different way in LangChain

#### Prompt #2

We do not have to write the whole input prompt again as we are tracking the conversation between the user and the model. So we can only write new message to the model

In [17]:
input_prompt="""No the output should be in jupyter notebook ipynb json format \
        and also add comments in the markdown cells of the jupyter notebook"""

We append the output response from the model to the messages and then again append the user input as HumanMessage

In [18]:

messages.append(res)

# make new query
messages.append(
    HumanMessage(content=input_prompt)
)

res = chat(messages)

print(res.content)


Here is the solution in Jupyter Notebook ipynb format with comments:

```
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Reading csv with Pandas\n",
    "\n",
    "In this notebook, we will read a csv file using Pandas and perform some basic data analysis on it."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importing necessary libraries\n",
    "import pandas as pd\n",
    "\n",
    "# Reading the csv file into a pandas dataframe\n",
    "matches_df = pd.read_csv('matches.csv')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Finding the list of unique cities where matches were played\n",
    "\n",
    "Let's find the list of unique cities where matches were played in the dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "

Great, that was much better in getting the desired output. Let's save the output so we can easily visualize the notebook

Saving the output to a New Notebook

In [19]:
create_notebook(result=res.content, filename="prompt_with_langchain")

Creating notebook: generated_notebooks/prompt_with_langchain.ipynb


## Generating Task Notebooks with LangChain

Now lets use LangChain to generate tasks instead of solutions.

In this case, we will give the model a topic on which it should create a excersie notebook.

In [20]:
system_template = SystemMessagePromptTemplate.from_template(
    'You are a helpful assistant. You create jupyter notebooks on specific tasks \
    asked by the user. The tasks you create should be solvable in python \
    You will always create a jupyter notebook in json format'
)

human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response}")

# create the list of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])

# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    input="Hi AI, How to read a csv file with pandas. Use jupyter notebook json format with comments",
    response=response_example
)
chat_prompt_value

ChatPromptValue(lc_kwargs={'messages': [SystemMessage(lc_kwargs={'content': 'You are a helpful assistant. You create jupyter notebooks on specific tasks     asked by the user. The tasks you create should be solvable in python     You will always create a jupyter notebook in json format', 'additional_kwargs': {}}, content='You are a helpful assistant. You create jupyter notebooks on specific tasks     asked by the user. The tasks you create should be solvable in python     You will always create a jupyter notebook in json format', additional_kwargs={}), HumanMessage(lc_kwargs={'content': 'Hi AI, How to read a csv file with pandas. Use jupyter notebook json format with comments', 'additional_kwargs': {}}, content='Hi AI, How to read a csv file with pandas. Use jupyter notebook json format with comments', additional_kwargs={}, example=False), AIMessage(lc_kwargs={'content': '{\n "cells": [\n  {\n   "attachments": {},\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n    "#

### Creating Notebook for Linear Regression task

#### 1st prompt

In [21]:
## keeping track of the conversation by appending the Human Message and AI response
messages = chat_prompt_value.to_messages()

## prompt 1
prompt = "Create a detailed step by step tasks in a jupyter notebook for applying linear regression on a dataset." 

messages.append(
    HumanMessage(content=prompt)
)

res = chat(messages)

print(res.content)

{
 "cells": [
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Applying Linear Regression on a Dataset"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 1: Importing Required Libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importing Required Libraries
",
    "import pandas as pd
",
    "import numpy as np
",
    "import matplotlib.pyplot as plt
",
    "from sklearn.linear_model import LinearRegression
",
    "from sklearn.model_selection import train_test_split"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 2: Loading the Dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Loading the Dataset
",
    "dataset = pd.read_csv('dataset.csv')"
   ]
  },
  {
   "cell_type": "markdown",
   "metada

On the first prompt we see that we got the response in the desired format but the response also has solutions to the tasks. 
We only need the model to create tasks for Linear Regression

So we ask the model again stating our requirements more clearly this time.

#### 2nd prompt

In [22]:
messages = chat_prompt_value.to_messages()

## prompt 3
prompt = """No I don't need solutions. I just need tasks questions which are 
    listed in the different markdown cells of the jupyter notebook 
    ipynb json format i.e. "cell_type":"markdown"
    """

messages.append(
    HumanMessage(content=prompt)
)

res = chat(messages)

print(res.content)

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Applying Linear Regression on a Dataset\n",
    "\n",
    "In this notebook, we will learn how to apply linear regression on a dataset using Python and Scikit-learn. We will follow the following steps:\n",
    "\n",
    "1. Importing the necessary libraries\n",
    "2. Loading the dataset\n",
    "3. Exploring the dataset\n",
    "4. Preprocessing the dataset\n",
    "5. Splitting the dataset into training and testing sets\n",
    "6. Creating a linear regression model\n",
    "7. Training the model\n",
    "8. Evaluating the model\n",
    "9. Making predictions\n",
    "10. Visualizing the results\n",
    "\n",
    "Let's get started!"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Importing the necessary libraries\n",
    "\n",
    "In this step, we will import the necessary libraries for applying linear regression on a dataset. We will be using the following

Great. Again after iterative prompting and being more specific and restrictive we finally got the desired output in correct format too.

Saving the output to the file

In [23]:
create_notebook(result=res.content, filename="generating_task_notebook.ipynb")

Creating notebook: generated_notebooks/generating_task_notebook.ipynb


## Evaluating Outputs

### Evaluating responses using rubric
- Rubric - a set of instructions or rules.

- Reference - [DeepLearningAI ChatGPT Course](https://learn.deeplearning.ai/chatgpt-building-system)

We define another helper function `get_completion_from_messages()`. This function will help us to get the response from the model with an input comprised of a list containing json like objects. These json like objects consists of system message and user message in a format `{'role': 'system', 'content': system_message}` and `{'role': 'user', 'content': user_message}` respectively.


In [24]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

We then define a function called `eval_vs_ideal()`. This function evaluates the response generated from the ChatGPT for a user query. 

This function takes 2 inputs:
- Ideal test case
- Response to evaluate

Specifically in this case, the function only checks if the response generated by the model suffices the requirements by comparing the answer to the ideal test case. 

The output from the model again goes into the model for evaluation, thereby leveraging the power of the LLMs.

The code compares the model response and the ideal answer (expert answer) and outputs from one of the following choices

Compare the factual content of the submitted answer with the expert answer. DO NOT ignore any differences in style or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. 
        Determine which case applies. Answer the question by selecting one of the following options: <br>
    (A) The submitted answer is a part of the expert answer and is fully consistent with it. <br>
    (B) The submitted answer contains all the same details as the expert answer. <br>
    (C) There is a disagreement between the submitted answer and the expert answer. <br>
    (D) The answers differ, but these differences don't matter from the perspective of factuality. <br>

In [25]:
def eval_vs_ideal(test_set, assistant_answer):

    ideal_user_msg = test_set['user_msg']
    ideal_answer = test_set['ideal_answer']
    
    system_message = """\
    You are an assistant that evaluates how well the AI Data Scientist agent \
    answers a user question by comparing the response to the ideal (expert) response
    """

    user_message = f"""\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {ideal_user_msg}
    ************
    [Expert]: {ideal_answer}
    ************
    [Submission]: {assistant_answer}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. DO NOT ignore any differences in style or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. 
        Determine which case applies. Answer the question by selecting one of the following options:
    (A) The submitted answer is a part of the expert answer and is fully consistent with it.
    (B) The submitted answer contains all the same details as the expert answer.
    (C) There is a disagreement between the submitted answer and the expert answer.
    (D) The answers differ, but these differences don't matter from the perspective of factuality.
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

> <span style="color:green"> **User Message** - </span> How to plot box plots with matplotlib. Do include cleaning of data before plotting and 
    add comments for the code. Also give the output in jupyter notebook ipynb "json" format.


Let us assume that the model gives us the below output when user message is same as above.

In [26]:

assistant_answer_1 = """
    # Importing necessary libraries
    import pandas as pd
    import matplotlib.pyplot as plt

    # Reading the data from a csv file
    data = pd.read_csv('data.csv')

    # Cleaning the data
    # Removing null values
    data.dropna(inplace=True)

    # Plotting the box plot
    plt.boxplot(data['column_name'])

    # Adding title and labels to the plot
    plt.title('Box plot of column_name')
    plt.xlabel('column_name')
    plt.ylabel('Values')

    # Displaying the plot
    plt.show()

    # Saving the plot as a png file
    plt.savefig('boxplot.png')
"""

Let us define the ideal test set, which consists of ideal user message and the ideal answer the user expects

In [27]:
test_set_ideal = {


    'user_msg':"""How to plot box plots with matplotlib.\
Do include cleaning of data before plotting and \
add comments for the code. Also give the output in \
jupyter notebook ipynb "json" format.""",



'ideal_answer':"""
    {
 "cells": [
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Plotting Boxplots with Matplotlib"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importing necessary libraries\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Reading the csv file into a pandas dataframe\n",
    "df = pd.read_csv('filename.csv')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cleaning the data\n",
    "# Removing null values\n",
    "df.dropna(inplace=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plotting the boxplot\n",
    "plt.boxplot(df['column_name'])\n",
    "plt.title('Boxplot of column_name')\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  },
  "orig_nbformat": 4
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
"""
}

With the model's response, we evaluate the response. If the evaluation is successful we can proceed with showing the response to the user, otherwise we can continue in loop to improve the answer until the evaluation is successful

In [28]:
print(eval_vs_ideal(test_set=test_set_ideal, assistant_answer=assistant_answer_1))

C) There is a disagreement between the submitted answer and the expert answer. 

Explanation: 
- The submitted answer includes additional steps such as adding title and labels to the plot, displaying the plot, and saving the plot as a png file, which are not present in the expert answer.
- The expert answer includes a step to read the csv file into a pandas dataframe, which is not present in the submitted answer.
- The expert answer specifies to plot the boxplot of a particular column, whereas the submitted answer does not specify which column to plot.


Now let us assume that after some iteration we get the response as below. Now let us check how the model evaluates this answer


In [29]:
assistant_answer_2 = """
 {
 "cells": [
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Plotting Boxplots with Matplotlib"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importing necessary libraries\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Reading the csv file into a pandas dataframe\n",
    "df = pd.read_csv('filename.csv')\n",
    "\n",
    "# Cleaning the data\n",
    "# Removing null values\n",
    "df.dropna(inplace=True)\n",
    "\n",
    "# Plotting the boxplot\n",
    "plt.boxplot(df['column_name'])\n",
    "plt.title('Boxplot of column_name')\n",
    "plt.show()"
   ]
  }
 ]
}

"""

In [30]:
print(eval_vs_ideal(test_set=test_set_ideal, assistant_answer=assistant_answer_2))

Answer: (A) The submitted answer is a part of the expert answer and is fully consistent with it. 

Explanation: The submitted answer is a subset of the expert answer and contains all the necessary details to plot box plots with Matplotlib. The code is identical to the expert answer, and the comments are also present. The only difference is that the submission has an extra blank line in the code, which does not affect the factuality of the answer. Therefore, the submitted answer is fully consistent with the expert answer.


## Thank You